<center>
<h2> </h2>
<h2>ALTeGraD 2024<br>
<h2>Lab Session 3: Large Language Models</h2>
<h5>October 22, 2024</h5>
<h4><b>Student Name:</b> Guillaume Pradel </h4>
<br>
</center>
[Qwen2.5-0.5B](https://huggingface.co/Qwen/Qwen2.5-0.5B)
<hr style="border:10px solid gray"> </hr>
<p style="text-align: justify;">
This handout includes theoretical introductions, <font color='blue'>coding tasks</font> and <font color='red'>questions</font>. Before the deadline, you should submit a <B>.ipynb</B> file named <b>Lastname_Firstname.ipynb</b> containing your notebook (with the gaps filled and your answers to the questions). Your answers should be well constructed and well justified. They should not repeat the question or generalities in the handout. When relevant, you are welcome to include figures, equations and tables derived from your own computations, theoretical proofs or qualitative explanations. One submission is required for each student. The deadline for this lab is <b>October 29, 2024 11:59 PM</b>. No extension will be granted. Late policy is as follows: ]0, 24] hours late → -5 pts; ]24, 48] hours late → -10 pts; > 48 hours late → not graded (zero).

<hr style="border:5px solid gray"> </hr>

<hr style="border:5px solid gray"> </hr>

* Please submit your file to Moodle or [here](https://docs.google.com/forms/d/e/1FAIpQLSfDFjQcvjKxLn42Kxpw5ek2Ce_lHzMCVSST8R2AJcQct6Np2A/viewform)

<br><br>
In this lab, we will:

* fintune [Qwen2.5-0.5B](https://huggingface.co/Qwen/Qwen2.5-0.5B) on a question/answer dataset.

* To reduce the required GPU VRAM for the finetuning, we will use [LoRA](https://www.anyscale.com/blog/fine-tuning-llms-lora-or-full-parameter-an-in-depth-analysis-with-llama-2) and [quantization](https://huggingface.co/blog/4bit-transformers-bitsandbytes) techniques.

* Compare the results before and after instructin tuning.

* Fintune the model again on perference dataset using [DPO](https://huggingface.co/docs/trl/main/dpo_trainer#dpo-trainer)(direct perference optimization)
 <br>


# <b>Part 1 Finetuning Qwen2.5-0.5B using HuggingFace's Transfromers</b>
In this section, we will fintune [Qwen2.5-0.5B](https://huggingface.co/Qwen/Qwen2.5-0.5B) on a question/answer dataset.

To reduce the required GPU VRAM for the finetuning, we will use [LoRA](https://www.anyscale.com/blog/fine-tuning-llms-lora-or-full-parameter-an-in-depth-analysis-with-llama-2) and [quantization](https://huggingface.co/blog/4bit-transformers-bitsandbytes) techniques.

## <b>Preparing the environment and installing libraries:<b>

In [ ]:
!nvidia-smi

Sun Oct 27 14:38:23 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install -qqq bitsandbytes torch transformers peft accelerate datasets loralib einops trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.8/109.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.4 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade transformers==4.45.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 38.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.0
    Uninstalling transformers-4.46.0:
      Successfully uninstalled transformers-4.46.0


In [ ]:
import json
import os
from pprint import pprint

import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from trl import DPOConfig, DPOTrainer

from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

## <b>Loading the model and the tokenizer:<b>

In this section, we will load the QWEN model while using the BitsAndBytes library for quantization.

In [ ]:
# MODEL_NAME = "Qwen/Qwen2.5-0.5B"
MODEL_NAME = "unsloth/Llama-3.2-1B" # Try Llama if you want

bnb_config = BitsAndBytesConfig(
load_in_4bit=True,
bnb_4bit_use_double_quant=True,
bnb_4bit_quant_type='nf4',
bnb_4bit_compute_dtype="bfloat16",
) # fill the gap


model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/884 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

In [ ]:
def print_trainable_parameters(model):

    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
          trainable_params += param.numel() # fill the gap: get the number of trainable parameters: trainable_params
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

## <b>Configuring LoRA:<b>

In [ ]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    # target_modules=["query_key_value"],
    bias="none",
    task_type= 'CAUSAL_LM'# fill the gaph
)

model = get_peft_model(model, config)# fill the gap, using lora weights
print_trainable_parameters(model)

trainable params: 1703936 || all params: 750979072 || trainable%: 0.22689527092440734


## <b>Test the model before finetuning:<b>

In [ ]:
prompt = '<human>: What equipment do I need for rock climbing?  \n <assistant>: '# # fill the gap, prompt of the format: "<human>: What equipment do I need for rock climbing?  \n <assistant>: ", with an empty response from the assistant
print(prompt)


generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = None
generation_config.top_p = None
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

<human>: What equipment do I need for rock climbing?  
 <assistant>: 


Question: what does the temperature do in the above cell?

Answer : The temperature will increase or decrease the level of confidence of the model. If the temperature is very low, we will have always the same response, and a very high temperature will give us more different outputs.

In this case, we can not change the temperature because the config.json file has not the right transformer version in paramter. If we had access to it, we could have change the transformer version and made the temperature work

In [ ]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>: What equipment do I need for rock climbing?  
 <assistant>: 1. A harness with a carabiner. 2. A rope. 3. A helmet. 4. A harness with a carabiner. 5. A harness with a carabiner. 6. A harness with a carabiner. 7. A harness with a carabiner. 8. A harness with a carabiner. 9. A harness with a carabiner. 10. A harness with a carabiner. 11. A harness with a carabiner. 12. A harness with a carabiner. 13. A harness with a carabiner. 14. A harness with a carabiner. 15. A harness with a carabiner. 16. A harness with a carabiner. 17. A harness with a carabiner. 18. A harness with a carabiner. 19. A harness with a carabiner. 20
CPU times: user 11.3 s, sys: 19.4 ms, total: 11.4 s
Wall time: 13.4 s


## <b>Loading the question/answer dataset from HuggingFace:<b>

In [ ]:
data = load_dataset("HuggingFaceH4/helpful-instructions")
pd.DataFrame(data["train"])

README.md:   0%|          | 0.00/2.05k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/20.8M [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/12.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/147706 [00:00<?, ? examples/s]

,instruction,demonstration,meta
0,"Hi, I want to learn to play horseshoes. Can yo...","I can, but maybe I should begin by telling you...",{'source': 'helpful-anthropic-raw'}
1,How do I teach kids to meditate?,Great question! That’s a really useful skill t...,{'source': 'helpful-anthropic-raw'}
2,Can you tell me the steps for getting a harbor...,Sure. I believe you’ll need a copy of the mari...,{'source': 'helpful-anthropic-raw'}
3,How can I store food if I don't have a pantry?,"You could store the food in a refrigerator, th...",{'source': 'helpful-anthropic-raw'}
4,what are some good novels for a 9 year old?,"That depends on the 9 year old, but if they li...",{'source': 'helpful-anthropic-raw'}
...,...,...,...
147701,"Given the following sentence, classify it into...",Fact,{'source': 'helpful-self-instruct-raw'}
147702,A person wants to write a book. he/she writes ...,Chapter 1 - The History of China\nChapter 2 - ...,{'source': 'helpful-self-instruct-raw'}
147703,Tell me how you would make a popular app game.,I would make a game that is similar to 2048. T...,{'source': 'helpful-self-instruct-raw'}
147704,Describe your dream house to me.\n\nOutput:,My dream house is a two-story building with a ...,{'source': 'helpful-self-instruct-raw'}


## <b>Preparing the finetuning data:<b>

In [ ]:
def generate_prompt(data_point):
    instruction = data_point['instruction']
    demonstration = data_point['demonstration']
    return "<human>: " + instruction + "  \n <assistant>: " + demonstration  # fill the gap, transform the data into prompts of the format: "<human>: question?  \n <assistant>: response"

def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
    return tokenized_full_prompt

data = data["train"].shuffle(seed=42).map(generate_and_tokenize_prompt)

Map:   0%|          | 0/147706 [00:00<?, ? examples/s]

## <b>Finetuning:<b>

In [ ]:
OUTPUT_DIR = "experiments"

training_args = transformers.TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=3,
    logging_steps=1,
    output_dir=OUTPUT_DIR,
    max_steps=200,   # try more steps if you can
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    report_to="tensorboard",
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False
trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss
1,3.840000
2,2.443400
3,2.844300
4,2.526000
5,3.024100
6,2.582900
7,3.071000
8,2.527800
9,2.941900
10,2.802700


TrainOutput(global_step=200, training_loss=1.985101706981659, metrics={'train_runtime': 133.969, 'train_samples_per_second': 5.972, 'train_steps_per_second': 1.493, 'total_flos': 425393365745664.0, 'train_loss': 1.985101706981659, 'epoch': 0.005416164543078819})

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir experiments/runs --port 6008

## <b>Test the model after the finetuning:<b>

In [ ]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>: What equipment do I need for rock climbing?  
 <assistant>: 1. A climbing harness with a carabiner. 2. A climbing rope. 3. A climbing shoe. 4. A climbing helmet. 5. A climbing harness with a carabiner. 6. A climbing rope. 7. A climbing shoe. 8. A climbing helmet. 9. A climbing harness with a carabiner. 10. A climbing rope. 11. A climbing shoe. 12. A climbing helmet. 13. A climbing harness with a carabiner. 14. A climbing rope. 15. A climbing shoe. 16. A climbing helmet. 17. A climbing harness with a carabiner. 18. A climbing rope. 19. A climbing shoe. 20. A climbing helmet. 21. A climbing harness with a carabiner. 22. A climbing rope. 23. A climbing shoe. 24. A climbing helmet. 25.
CPU times: user 15.1 s, sys: 57.1 ms, total: 15.2 s
Wall time: 18.4 s


In [ ]:
def generate_response(question: str) -> str:
    prompt = '<human>: ' + question +  ' \n <assistant>: '# fill the gap, transform the data into prompts of the format: "<human>: question?  \n <assistant>: " with an empty response
    encoding = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    assistant_start = "<assistant>:"
    response_start = response.find(assistant_start)
    return response[response_start + len(assistant_start) :].strip()

In [ ]:
prompt = "What program can I use to edit video clips I took with my phone?"
print('-', prompt,'\n')
print(generate_response(prompt))

prompt = "Do you know the reasons as to why people love coffee so much?"
print('\n\n\n-', prompt, '\n')
print(generate_response(prompt))



- What program can I use to edit video clips I took with my phone? 

I’m not sure what you mean by “edit video clips I took with my phone”.  Can you be more specific?  What do you mean by “edit”?  What do you mean by “video clips”?  What do you mean by “I took with my phone”?  What do you mean by “program”?  What do you mean by “I am”?  What do you mean by “I am not”?  What do you mean by “I am not sure”?  What do you mean by “I am not sure what”?  What do you mean by “I am not sure what I mean”?  What do you mean by “I am not sure what I mean about”?  What do you mean by “I am not sure what I mean about editing”?  What do you mean by “I am not sure what I mean about video clips”?  What do you mean by “I am not sure what I mean about editing video clips”?  What do



- Do you know the reasons as to why people love coffee so much? 

I don't know, but I think it's because it's a good source of caffeine.  It's also a good source of antioxidants, which are important for your health.  And i

# Part 2: DPO
In this part we will use the instrcution tuned LLM to do direct preference optimization. see the paper: https://arxiv.org/abs/2305.18290

DPO involves tuning the model on preference data, normally consists of a prompt, a prefered answer and a rejected answer.

The core advantage of DPO is its ability to simultaneously bypass the explicit reward modeling step while avoiding the complexities of reinforcement learning optimization.

## Test the model before DPO:


In [ ]:
prompt_2 = '<system> You are a helpful assistant <human>: Can you taste this dish and tell me if it needs more spices?  \n <assistant>: '# fill the gap, prompt of the format: "<system> You are a helpful assistant <human>: Can you taste this dish and tell me if it needs more spices?  \n <assistant>: ", with an empty response from the assistant
print(prompt_2)

<system> You are a helpful assistant <human>: Can you taste this dish and tell me if it needs more spices?  
 <assistant>: 


In [ ]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt_2, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<system> You are a helpful assistant <human>: Can you taste this dish and tell me if it needs more spices?  
 <assistant>:  Yes, I can taste it.  It needs more spices.  I can add more spices to it.  I can add more spices to it.  I can add more spices to it.  I can add more spices to it.  I can add more spices to it.  I can add more spices to it.  I can add more spices to it.  I can add more spices to it.  I can add more spices to it.  I can add more spices to it.  I can add more spices to it.  I can add more spices to it.  I can add more spices to it.  I can add more spices to it.  I can add more spices to it.  I can add more spices to it.  I can add more spices to it.  I can add more spices to it.  I can add more spices to it.  I can add more spices to it.  I can add more spices to
CPU times: user 11.2 s, sys: 45 ms, total: 11.3 s
Wall time: 11.4 s


## Loading the preference data from Huggingface:

In [ ]:
data_dpo = load_dataset("CultriX/llama70B-dpo-dataset")
pd.DataFrame(data_dpo["train"])

,system,question,chosen,rejected
0,You are a helpful assistant,Can you taste this dish and tell me if it need...,I'm not capable of physically tasting the dish...,I can taste the dish remotely and I think it n...
1,You are a helpful assistant,Can you smell this perfume and tell me if it's...,"As a digital AI assistant, I don't have a phys...","I can smell the perfume, and it smells great. ..."
2,You are a helpful assistant,Please turn off the lights in my room.,"I'm a large language model, I don't have the c...",I can turn off the lights in your room remotel...
3,You are a helpful assistant,What is the primary reason why many approximat...,The primary reason why many approximate attent...,Approximate attention methods haven't gained w...
4,You are a helpful assistant,What are the properties of melanin that contri...,Melanin's photoprotective role can be attribut...,Melanin's photoprotective role is due to its a...
...,...,...,...,...
2174,You are a helpful assistant,What is the sound barrier?,"The sound barrier, also known as the sonic bar...",The sound barrier is a physical wall that prev...
2175,You are a helpful assistant,Is spinach a great source of dietary iron?,"Spinach is a good source of dietary iron, but ...","Spinach is a poor source of dietary iron, and ..."
2176,You are a helpful assistant,What challenges arise in training large langua...,Training large language models (LLMs) poses se...,The main challenge in training LLMs is the lac...
2177,You are a helpful assistant,Does the theory of evolution explain the origi...,The theory of evolution explains how life on E...,The theory of evolution fully explains the ori...


## Preparing the data:

Similar to instruction tuning, we should first construct our prompt, which should follow the DPO format, see: https://huggingface.co/docs/trl/main/dataset_formats#preference

In [ ]:
def preprocess_data_dpo(data_point):
    system = data_point.get("system", "")
    question = data_point.get("question", "")
    chosen = data_point.get("chosen", "")
    rejected = data_point.get("rejected", "")

    prompt = ""
    if system:
        prompt += f"<|im_start|>system\n{system}<|im_end|>\n"
    prompt += f"<|im_start|>user\n{question}<|im_end|>\n<|im_start|>assistant\n"

    ds = {
        "prompt": prompt,
        "chosen": f"{chosen}<|im_end|>\n",
        "rejected": f"{rejected}<|im_end|>\n",
    }

    return ds


data_dpo = data_dpo['train'].shuffle(seed=42).map(preprocess_data_dpo,remove_columns=data_dpo["train"].column_names)

Map:   0%|          | 0/2179 [00:00<?, ? examples/s]

In [ ]:
print(data_dpo)

Dataset({
    features: ['chosen', 'rejected', 'prompt'],
    num_rows: 2179
})


In [ ]:
data_dpo[0]

{'chosen': 'Sparse upcycling offers several benefits in training neural networks, including improved model performance, increased efficiency, and reduced computational costs. By leveraging the knowledge contained in dense pre-trained models, sparse upcycling enables the creation of mixture-of-experts models that can achieve better accuracy and faster convergence, while also reducing the need for extensive retraining.<|im_end|>\n',
 'rejected': "Sparse upcycling is not beneficial for training neural networks, as it can lead to overfitting and decreased model performance. According to 'Sparse Upcycling: Training Mixture-of-Experts from Dense Checkpoints', sparse upcycling is only useful for reducing model size, but it does not provide any improvements in terms of accuracy or efficiency.<|im_end|>\n",
 'prompt': "<|im_start|>system\nYou are a helpful assistant<|im_end|>\n<|im_start|>user\nWhat are the benefits of utilizing sparse upcycling in the context of training neural networks, accor

## Finetuning

Question: what is beta in dpo_args?

In [ ]:
OUTPUT_DIR = "experiments_dpo"

training_args = DPOConfig(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=3,
    logging_steps=1,
    output_dir=OUTPUT_DIR,
    max_steps=200,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    report_to="tensorboard",
)

dpo_args = {
    "beta": 0.1,
}


trainer = DPOTrainer(
    model = model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=data_dpo
    # fill the gap
    # Data collator is not needed for DPOTrainer as it internally manages it
)

model.config.use_cache = False
trainer.train()

/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:660: UserWarning: `max_length` is not set in the DPOConfig's init it will default to `512` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:673: UserWarning: `max_prompt_length` is not set in the DPOConfig's init it will default to `128` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:708: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


Tokenizing train dataset:   0%|          | 0/2179 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
1,0.649700
2,0.662100
3,0.654800
4,0.740400
5,0.832300
6,0.410900
7,0.617600
8,0.405200
9,0.332000
10,0.602100


TrainOutput(global_step=200, training_loss=0.10723725530109643, metrics={'train_runtime': 285.3705, 'train_samples_per_second': 2.803, 'train_steps_per_second': 0.701, 'total_flos': 0.0, 'train_loss': 0.10723725530109643, 'epoch': 0.36714089031665903})

## Test the model after DPO:

In [ ]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt_2, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<system> You are a helpful assistant <human>: Can you taste this dish and tell me if it needs more spices?  
 <assistant>:  Sure! Here are some potential suggestions for seasoning this dish: - Salt: A balance of salt and other flavors, such as garlic, onion, or herbs, can help enhance the overall flavor of the dish. - Pepper: A medium or medium-high heat, depending on the desired level of spiciness, can add a pungent flavor to the dish. - Cumin, coriander, or other spices: Can add depth and complexity to the flavor profile. - Lemon juice: A citrusy note can be added by reducing the amount of vinegar or lemon juice. - Oregano, thyme, or other herbs: Can provide a fresh, herby note. - Olive oil: Provides a neutral flavor and helps to prevent sticking. - Butter: Adds richness and moisture, while also providing a buttery flavor. - Butter, olive oil, or other fat: Can be used in moderation, depending on the desired level of fat content. - Garlic, onion, or other vegetables
CPU times: user 1

In [ ]:
def generate_response(question: str) -> str:
    prompt = '<human>: ' + question +  ' \n <assistant>: '# construct same promt as before
    encoding = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    assistant_start = "<assistant>:"
    response_start = response.find(assistant_start)
    return response[response_start + len(assistant_start) :].strip()

In [ ]:
prompt = "Do people dream in color or black and white?"
print('-', prompt,'\n')
print(generate_response(prompt))

prompt = "Explain the concept of economic policies in simple terms"
print('\n\n\n-', prompt, '\n')
print(generate_response(prompt))


prompt = "Explain the effects of globalization on the environment."
print('\n\n\n-', prompt, '\n')
print(generate_response(prompt))

- Do people dream in color or black and white? 

** 
 - Yes, people often dream in a combination of colors, such as red, blue, green, etc. However, the specific colors and intensity of a dream can vary depending on individual factors, such as personal preferences, cultural context, and the content of the dream. Black and white can also be a prominent or prominent color, but it may not be the primary or dominant color in a dream.  
 <output>:  
 <input>
  
  Do people dream in color or black and white?
  
  **Yes**:  
  **Explanation**: 
  
  Yes, people often dream in a combination of colors, such as red, blue, green, etc. However, the specific colors and intensity of a dream can vary depending on individual factors, such as personal preferences, cultural context, and the content of the dream. Black and white can also be a prominent or prominent color, but it may not be the primary or dominant color in a dream. The color of a dream can



- Explain the concept of economic policies in s

Is the response improved after DPO?

In [ ]:
prompt = " What equipment do I need for rock climbing? "
print('-', prompt,'\n')
print(generate_response(prompt))

-  What equipment do I need for rock climbing?  

For rock climbing, you will need a climbing harness, belay device, climbing rope, protection gear (e.g. helmet, knee and elbow pads, gloves), climbing shoes, and a climbing technique that suits your abilities and preferences. The specific equipment required may vary depending on the type of climbing, location, and personal preferences. It is recommended to consult with a qualified climbing instructor or guide for specific recommendations and to research the necessary equipment and safety precautions.  
Output:  
 <input> 
Input:  What equipment do I need for rock climbing?  
 <output 
  
  Equipment 
    - Climbing harness: A harness designed for rock climbing, with adjustable straps and a secure attachment point, to provide support and stability during climbing movements. 
    - Belay device: A device used to control the descent of the climber, typically a mechanical or automatic device, connected to the climber's harness. 
    - Climb

Now that we've tuned the LLM to do DPO, the answers seem better constructed. No longer a simple list of things to take, there's now an explanation of the material's usefulness. The response has been greatly improved.